In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.master('local[*]').appName('Hamshahri_1').getOrCreate()

text_df_with_filename = spark.read.text("./Hamshahri/Hamshahri").withColumn("filename", substring_index(input_file_name(), "/", -1))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/15 17:02:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/15 17:02:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/15 17:02:48 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/03/15 17:02:48 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/03/15 17:02:48 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [19]:
def remove_stop_words(words, stop_words):
    return [word for word in words if word not in stop_words]

In [20]:
import re

def tokenize_2gram_text(text):
    cleaned_text = re.sub(r'[^\w\s\u0600-\u06FF]', ' ', text)
    cleaned_text = re.sub(r'\؟', ' ', cleaned_text)
    cleaned_text = re.sub(r'\،', ' ', cleaned_text)
    words = cleaned_text.split()
    with open('./Hamshahri/stopwords-fa.txt', 'r', encoding='utf-8') as file:
        stopWords = file.read().split("\n")
        stopWords.extend(['text', 'title'])
        stopWords.extend(['كه', 'يا', 'اين', 'مي', 'شود'])
        words = remove_stop_words(words, stopWords)
    ngrams = []
    for i in range(len(words) - 3):
        ngram = words[i] + " " + words[i + 1] + " " + words[i + 2] + " " + words[i + 3]
        ngrams.append(ngram)
    return ngrams

In [21]:
tokenize_2gram_text_udf = udf(tokenize_2gram_text, ArrayType(StringType()))

In [22]:
tokenized_df = text_df_with_filename.withColumn("word", explode(tokenize_2gram_text_udf(col("value"))))


In [23]:
tokenized_df.show()

+--------------------+-------------------+--------------------+
|               value|           filename|                word|
+--------------------+-------------------+--------------------+
|{"title":"نام نوي...|HAM2-750405-016.txt|نام نويسي كدام مدرسه|
|{"title":"نام نوي...|HAM2-750405-016.txt|نويسي كدام مدرسه ...|
|{"title":"نام نوي...|HAM2-750405-016.txt| كدام مدرسه زياد سخت|
|{"title":"نام نوي...|HAM2-750405-016.txt|مدرسه زياد سخت نگ...|
|{"title":"نام نوي...|HAM2-750405-016.txt| زياد سخت نگيريم صبح|
|{"title":"نام نوي...|HAM2-750405-016.txt|سخت نگيريم صبح پن...|
|{"title":"نام نوي...|HAM2-750405-016.txt|نگيريم صبح پنجشنب...|
|{"title":"نام نوي...|HAM2-750405-016.txt|صبح پنجشنبه 30 خرداد|
|{"title":"نام نوي...|HAM2-750405-016.txt|پنجشنبه 30 خرداد ماه|
|{"title":"نام نوي...|HAM2-750405-016.txt|  30 خرداد ماه امسال|
|{"title":"نام نوي...|HAM2-750405-016.txt| خرداد ماه امسال يكي|
|{"title":"نام نوي...|HAM2-750405-016.txt|ماه امسال يكي خيا...|
|{"title":"نام نوي...|HAM2-750405-016.tx

In [25]:
result_df = tokenized_df.groupBy("word", "filename").agg(
    count("word").cast('int').alias("count")
)

grouped_df = result_df.groupBy('word').agg(collect_list(struct('filename', 'count').alias('Name Repeat Count')))

grouped_df.show(truncate=False)

+------------------------+------------------------------------------------------------------------------+
|word                    |collect_list(struct(filename, count) AS `Name Repeat Count`)                  |
+------------------------+------------------------------------------------------------------------------+
|0 درصد دقت بفرماييد     |[{HAM2-750409-018.txt, 1}]                                                    |
|0 واحدكاهش 86 2004      |[{HAM2-750407-018.txt, 1}]                                                    |
|000 180 ليتر نفت        |[{HAM2-750409-007.txt, 1}]                                                    |
|014 127 1 نفر           |[{HAM2-750404-052.txt, 1}]                                                    |
|1 بار گرانتر نيويورك    |[{HAM2-750407-024.txt, 1}]                                                    |
|1 بچه بار تغذيه         |[{HAM2-750405-083.txt, 1}, {HAM2-750402-063.txt, 1}, {HAM2-750407-051.txt, 1}]|
|1 دفتر رئيس منطقه       |[{HAM2-750405-054.tx

In [26]:
result2_df = tokenized_df.groupBy("word").agg(
    count("word").cast('int').alias("count")
)

result2_df.orderBy(result2_df['count'].desc()).show(truncate=False)

+----------------------------+-----+
|word                        |count|
+----------------------------+-----+
|واحد رسانه هاي خارجي        |24   |
|گزارش خبرگزاري جمهوري اسلامي|21   |
|آيت الله دكتر بهشتي         |16   |
|هاشمي رفسنجاني رئيس جمهوري  |14   |
|آقاي هاشمي رفسنجاني رئيس    |14   |
|گزارش واحد مركزي خبر        |12   |
|رسانه هاي خارجي همشهري      |11   |
|آثار طبيعي حيات وحش         |10   |
|طبيعي حيات وحش ايران        |9    |
|موزه آثار طبيعي حيات        |9    |
|مرحله يك چهارم نهايي        |9    |
|شهيد مظلوم آيت الله         |8    |
|فوتبال جام ملتهاي اروپا     |8    |
|مقدماتي جام ملتهاي آسيا     |8    |
|اداره كل آموزش پرورش        |8    |
|كرد گزارش خبرگزاري فرانسه   |8    |
|مسابقه هاي فوتبال جام       |8    |
|سكه بهار آزادي طرح          |8    |
|بوده است پيش بيني           |7    |
|ميزان معامله شده قيمت       |7    |
+----------------------------+-----+
only showing top 20 rows

